In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install gradio

In [ ]:
!nvidia-smi

Sun Nov  9 14:37:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import gradio as gr


def greet(grt):
    return "Hello " + grt


demo = gr.Interface(fn=greet, inputs="text", outputs="text")

demo.launch(share=True)

# flag button: Save inputs outputs in a csv file.
# report or save examples of bad, incorrect, or interesting outputs from your app.

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f9ea87f81901686d0a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# rename input box
import gradio as gr

def greet(name):
    return "Hello " + name


# We instantiate the Textbox class
textbox = gr.Textbox(label="Type your name here:", placeholder="John Doe", lines=2)

gr.Interface(fn=greet, inputs=textbox, outputs="text").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d435dead05fdb2649c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import librosa
import numpy as np
import io

# Build a speech recongination demo
# 1. setup model, using model from huggingface inference api to save calculation resource
import os
from huggingface_hub import InferenceClient

import tempfile # create files in /tmp
import soundfile as sf

# make sure you have set your HF_TOKEN in colab env
# speech recognition need provider huggingface, default provider sambanova didn't support this task
stt_client = InferenceClient(model="openai/whisper-large-v3", provider="hf-inference")

def speech_to_text(audio):
  if audio is None:
    return "No audio received"
  sr, data = audio
  if data.dtype != np.float32:
        data = data.astype(np.float32) / np.max(np.abs(data))
  target_sr = 16000
  data_16k = librosa.resample(data, orig_sr=sr, target_sr=target_sr)
  transed_audio = (target_sr, data_16k)

  # save to a temporary WAV file to fit the input format of stt_client
  with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
      sf.write(tmp.name, data_16k, target_sr)
      tmp_path = tmp.name

  # using inference client to get the prediction
  result = stt_client.automatic_speech_recognition(audio=tmp_path)
  # result = {"text":"debugging"}
  return sr, transed_audio, result["text"]

speech_rate_box = gr.Textbox(label="Original Speech Rate: ")
audio_16k = gr.Audio(label="Audio converted to 16000")
speech_text = gr.Textbox(label="Recognize result: ")

demo = gr.Interface(fn=speech_to_text,
             inputs = gr.Audio(sources=["microphone"], type="numpy", label="Speak here..."),
             outputs = [
                 speech_rate_box,
                 audio_16k,
                 speech_text
             ]
             )

# if the mic couldn't be turn on, open the demo with the public url to open your mic
# demo.launch(debug=True, share=True)
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5b07f3366764326e2d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# With blocks
# A demo that convert text to audio
import gradio as gr

# create pipeline
from transformers import pipeline

import soundfile as sf
import tempfile

model = pipeline("text-to-speech")

def text_to_speech(text:str):
  result = model(text)
  audio_array = result["audio"]
  sr = result["sampling_rate"]

  # (num_samples, 1)
  audio_array = np.array(audio_array).flatten()

  return (sr, audio_array)

demo = gr.Blocks()

with demo:
  gr.Markdown(
      """
      # Text to Speech
      Type and listen!
      """
  )
  input = gr.Textbox(label="Input the text:",
                     value="Hello there",
                     lines=2)
  submit = gr.Button("submit")
  audio = gr.Audio()

  submit.click(fn = text_to_speech,
               inputs=input,
               outputs=audio)

demo.launch()

No model was supplied, defaulted to suno/bark-small and revision 1dbd7a1 (https://huggingface.co/suno/bark-small).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b1d71c70dfe0fefe74.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
